# ThinCurr Python Example: XML Creation 

In this notebook we demonstrate how to utilize the filaments.py script from the ThinCurr package to help create oft xml files for user defined coils. 

## Load ThinCurr library

To load the ThinCurr filaments python script, you will need to define where ThinCurr is located. This can be done through the PYTHONPATH enviornment variable or within the script using sys.path.append() as shown below, where we look for the environment variable OFT_ROOTPATH to procide the path to where the OpenFUSIONToolkit is installed. If this enviornment variable is not defined, you can do so within the script as well by setting:
 
 
 ```os.environ["OFT_ROOTPATH"] = \path\to\OpenFUSIONToolkit\install_release```


In [ ]:
import os 
import sys
thincurr_python_path = os.getenv('OFT_ROOTPATH')
if thincurr_python_path is not None:
    sys.path.append(os.path.join(thincurr_python_path,'python'))
from OpenFUSIONToolkit.ThinCurr.filaments import Icoil, Vcoil, create_oft

## Creating a basic coil array
You can load data through a text file that is in the format of Coil Name \linebreak R \n Z. Where \n is a space. This defines coils, where each is a set of (R,Z) filments assuming axisymmetry. See fcoils_pts_test.txt to see the specified format. If you wanted to represent a singular filament, you would only need a single R Z point. Names can be left blank when defining an Icoil or Vcoil class 

You may have any kind of input you want as long as it can interface with initializing the classes (Icoil or Vcoil classes) correctly. 

In [ ]:
data = {}

with open("fcoil_pts_test.txt", "r") as f:
    lines = f.readlines()

current_name = None
for line in lines:
    line = line.strip()
    if not line:
        continue  # skip blank lines

    parts = line.split()
    if len(parts) == 1:
        # It's a name
        current_name = parts[0]
        data[current_name] = {'radius': [], 'Z': []}
    elif len(parts) == 2:
        # It's an r z pair
        r, z = map(float, parts)
        data[current_name]['radius'].append(r)
        data[current_name]['Z'].append(z)
    else:
        raise ValueError(f"Unexpected line format: {line}")


In [ ]:
coil_list = [Icoil(name, values['R'], values['Z']) for name, values in data.items()]
resistivites = [1E-4]
create_oft('fcoil.xml', coil_list, [], resistivites) 

oft file created


## Creating a custom coil xml file

You can also create a custom coil that does not assume axisymmetry, however you must define it as a set of npoints defined by xyz coordinates. You can also set parameters like the restivity per length 

In [ ]:
data = {} 
npoints = 0 
with open("custom_coil.txt", "r") as f:
    lines = f.readlines()
name = None 
for line in lines: 
    line = line.strip()
    if not line:
        continue  # skip blank lines
    parts = line.split()
    if len(parts) == 1:
        # it's a name
        name = parts[0]
        data[name] = {'xyz': []}
    elif len(parts) == 3:
        # it's an x y z point 
        x, y, z = map(float, parts)
        data[name]['xyz'].append((x, y, z))
        npoints +=1 
    else:
        raise ValueError(f"Unexpected line format: {line}")


In [ ]:
coil_list = [Vcoil(name, R=10, Z=None, xyz=values['xyz'], scale=0, resistivity_per_length=0.01, sens_mask=1, npoints=npoints, iscustom=True) for name, values in data.items()] 
create_oft('custom_coil.xml', [], coil_list, resistivites) 

custom1 and 10 and [(1.0, 2.0, 3.0), (4.0, 5.0, 6.0), (7.0, 8.0, 9.0)] and 0.01 and 1 and 3 and 0
oft file created
